In [1]:
import os
from multiprocessing import Pool

import networkx as nx
import numpy as np
from pprint import pprint

from src.vrp_study.pdptw_model.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from src.vrp_study.pdptw_model.routing_model import find_optimal_paths
from src.vrp_study.data_loader import parse_data
from src.vrp_study.configs import ModelConfig
from src.vrp_study.data_model import Tariff, Cargo, Node
from src.vrp_study.routing_manager import RoutingManager
from src.vrp_study.pdptw_model.solution_builder import SolutionBuilder
import pickle

In [2]:
def func(du, dv):
    return np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2)


def calc(data):
    u, du, p2coordinate = data
    return {(u, v): func(du, dv) for v, dv in p2coordinate.items()}


def build_routing_manager(
        depo: Node,
        cargos: list[Cargo],
        tariff: Tariff,
        p: Pool
) -> RoutingManager:
    p2coordinates = {}

    p2coordinates.update({
        crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
    })
    p2coordinates[depo.id] = depo.coordinates

    res = list(p.imap_unordered(calc, [(u, du, p2coordinates) for u, du in p2coordinates.items()]))
    distance_matrix = {}
    time_matrix = {}
    for r in res:
        for k, v in r.items():
            distance_matrix[k] = v
            time_matrix[k] = v

    routing_manager = PDRoutingManagerBuilder(
        distance_matrix=distance_matrix,
        time_matrix=time_matrix,
        model_config=ModelConfig(max_execution_time_minutes=1)
    )

    routing_manager.add_cargos(cargos)
    routing_manager.add_tariff(tariff)

    routing_manager.add_depo(depo)

    routing_manager = routing_manager.build()
    return routing_manager

In [3]:
def graph_to_real_node(g: nx.DiGraph, rm: RoutingManager) -> nx.DiGraph:
    res = nx.DiGraph()
    for u, v, d in g.edges(data=True):
        a = rm.nodes()[u].routing_node.id
        b = rm.nodes()[v].routing_node.id
        res.add_edge(a, b, **d)
    return res

In [4]:
import uuid
def rms_calc(data):
    routing_manager, benchmark_type, name = data
    routing_manager: RoutingManager = routing_manager
    sb = SolutionBuilder()
    cg = sb.generate_full_graph(routing_manager)

    with open(f'../data/Li & Lim benchmark/graphs/graph_{benchmark_type}_{name}.pkl', 'wb') as f:
        pickle.dump(graph_to_real_node(cg, routing_manager), f)

    sol = find_optimal_paths(routing_manager, sb)
    sols = []
    
    for s in sol[0]:
        if len(s) > 0:
            sols.append([routing_manager.nodes()[i].routing_node.id for i in s[1:-1]])
    with open(f'../data/Li & Lim benchmark/parsed_solutions/{benchmark_type}/{str(uuid.uuid4())}_{name}', 'wb') as f:
        pickle.dump((sols, routing_manager), f)
    del routing_manager, benchmark_type, name, sol, sols


In [5]:
from tqdm import tqdm
from loguru import logger as log

log.remove()
NUM_WORKERS = 20
MAX_SIZE = max(10, NUM_WORKERS)  # создаем столько менеджеров. потом в парралель NUM_WORKERS  решают

with Pool(NUM_WORKERS) as p:
    for benchmark_type in ['pdp_100']:  #os.listdir('../data/Li & Lim benchmark'):
        rms = []
        for name in tqdm(os.listdir(f'../data/Li & Lim benchmark/benchmarks/{benchmark_type}')):
            depo, cargos, tariff = parse_data(f'../data/Li & Lim benchmark/benchmarks/{benchmark_type}/{name}')
            rms.append((build_routing_manager(depo, cargos, tariff, p), benchmark_type, name))
            if len(rms) == MAX_SIZE:
                rr = list(p.imap_unordered(rms_calc, rms))
                rms = []
        break
    if len(rms) > 0:
        rr = list(p.imap_unordered(rms_calc, rms))
        rms = []

100%|██████████| 56/56 [00:49<00:00,  1.13it/s]


In [ ]:
from src.vrp_study.pdptw_model.solution_checker import check_solution

for file_name in os.listdir('../data/graphs'):
    solution_name = file_name.replace('_cg', '')
    with open(f'../data/results/{solution_name}', 'rb') as f:
        res = pickle.load(f)
    sols, rm = res
    rm: RoutingManager = rm
    assert check_solution(rm, sols)

In [ ]:

for file_name in os.listdir('../data/graphs'):
    solution_name = file_name.replace('_cg', '')
    print(file_name, solution_name)
    with open(f'../data/graphs/{file_name}', 'rb') as f:
        cg = pickle.load(f)
    with open(f'../data/results/{solution_name}', 'rb') as f:
        res = pickle.load(f)
    sols, rm = res
    rm: RoutingManager = rm
    cg: nx.DiGraph = cg
    # print(cg.edges())

    edges2remove = set()

    for u, v, d in cg.edges(data='length'):
        if d > 7:
            edges2remove.add((u, v))
    cg.remove_edges_from(edges2remove)

    for sol in sols:
        cargo_path_set = set()
        cargo_path = []
        for i in range(len(sol)):
            pdp = rm.nodes()[sol[i]].pdp_id
            if pdp not in cargo_path_set:
                cargo_path_set.add(pdp)
                cargo_path.append(pdp)
        print(sol)
        print(cargo_path)

        for i in range(len(cargo_path) - 1):
            p1 = cargo_path[i]
            p2 = cargo_path[i + 1]
            n1 = rm.get_pick_up_and_delivery_nodes()[p1][0]
            n2 = rm.get_pick_up_and_delivery_nodes()[p2][0]
            e = cg.edges()[n1, n2]
            print(e)

In [ ]:
cg

In [ ]:
from matplotlib import pyplot as plt

edge_values = np.array([d['length'] for u, v, d in cg.edges(data=True)])
edge_values = edge_values[edge_values < 7]
plt.hist(edge_values, bins=100)

In [ ]:
edge_values.min(), edge_values.max()